In [1]:
import pandas as pd
import os
from time import ctime
pd.options.display.max_rows=300

In [2]:
%%time
fl=pd.read_csv('./further_clean_train.csv', chunksize=10000)
df = pd.concat([i for i in fl], ignore_index=True)
print df.shape

/home/shj16110/anaconda2/lib/python2.7/site-packages/IPython/core/magic.py:188: DtypeWarning: Columns (5,9,13,28,45,46,50,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


(195186, 86)
CPU times: user 3.23 s, sys: 282 ms, total: 3.51 s
Wall time: 3.5 s


In [5]:
df[df['username']=='dabitray']['about.profile_description'].values

array([ '>Photoshop,\n> Web Search, \n> Web Scraping,\n>Creating Facebook Groups,\n> Fan Page,\n>Like Page,\n>Social Networking,\n>Product Design,\n>Net,\n>Data Entry,\n>Big Data,\n>Article Submission,\n>Data Processing,\n>Copy Typing\n>Excel,\n>Word,\n>PDF Etc\nI am an informatics student with plenty of free time that I would like to fill with additional work. I am a responsible person who takes her job seriously and sticks to agreed deadlines, plans and commitments.\nAny work I sign in for, I will fulfill according to agreement.\n\nDabit Ray'], dtype=object)

In [6]:
df[df['username']=='stefanfulton']['about.profile_description'].values

array([ 'STILL SEEKING FIRST FREELANCER GIG, REVIEWS & COMMENTS!!! \n(Get me for cheap while you can!)\n\nExpert multidisciplinary design and marketing manager, with five years professional photography & videography experience, over ten years editing experience using the full suite of Adobe programs, and nearly four years experience marketing, editing, and writing for small business. With an aim to please, Stefan strives to make himself a valued asset to each and every client, no matter the size of the job. Combining a strong work ethic and quick turnover rate, Stefan will do everything he can to exceed your expectations, as the goal is always to form a lasting and harmonious relationship. Make the smart move and get in contact now!'], dtype=object)

In [9]:
def helper(text):
    try:
        text=text.lower()
        sp=text.split()
        m=sp.count('he')+sp.count('his')
        f=sp.count('she')+sp.count('her')
        if f>0:
            return 'F'
        elif m>0:
            return 'M'
        else:
            return None
    except:
        return None

In [73]:
%%time
df['desc_stat'] = df['about.profile_description'].apply(helper)

CPU times: user 1.07 s, sys: 13 ms, total: 1.08 s
Wall time: 1.06 s


In [75]:
df['desc_stat'].value_counts()

M    1208
F     279
Name: desc_stat, dtype: int64

___

In [88]:
label = pd.read_csv('./New_Training_set.csv')
label.shape

(66593, 2)

In [89]:
label.head()

,label,username
0,1,billion
1,1,ArmenSoft
2,1,okbookman
3,1,revitupmedia
4,1,AnsonSequeira


In [91]:
label['username']=label['username'].str.lower()
df['username']=df['username'].str.lower()

In [92]:
df_t = df.set_index('username').join(label.set_index('username'), how='inner')
df_t.shape

(55640, 87)

In [93]:
df_t['label'].value_counts()

1    45753
0     9887
Name: label, dtype: int64

In [98]:
df_t.shape

(55640, 87)

In [96]:
df_t[['desc_stat','label']].head()

,desc_stat,label
username,,
a10110a,None,1
a15386,None,1
a1anm,None,1
a1bert,None,1
a1empire,None,1


In [99]:
df_t['desc_stat'].value_counts()

M    301
F     96
Name: desc_stat, dtype: int64

In [101]:
df_n = df_t[df_t['desc_stat'].notnull()]
df_n.shape

(397, 87)

In [106]:
df_n = df_n[['desc_stat','label']]
df_n.head()

,desc_stat,label
username,,
a2ksoft,M,1
aardvarkfilms,M,1
abdoabdalla,M,1
abidmansoor,M,1
adhaytma3010,M,1


In [127]:
df_nn = df_n.applymap(lambda x: {'M':1,1:1,'F':0,0:0}[x])
df_nn.head()

,desc_stat,label
username,,
a2ksoft,1,1
aardvarkfilms,1,1
abdoabdalla,1,1
abidmansoor,1,1
adhaytma3010,1,1


In [128]:
diff = df_nn[df_nn['desc_stat']!=df_nn['label']]
diff.shape

(43, 2)

In [129]:
diff.head()

,desc_stat,label
username,,
anghelvioleta,1,0
anomakit,1,0
anomakit,1,0
babywebmaster,0,1
bmihai10,1,0


___

In [130]:
gd = pd.read_csv('./users.golden_label.csv')
gd.shape

(168457, 8)

In [131]:
gd.head()

,user_id,username,freq,male.prob,fname.gender,male.prob2,fname.gender2,pic.gender
0,5,billion,2.0,1.0000,male,0.920000,male,NaN
1,63,paddyshyam,NaN,NaN,NaN,NaN,NaN,NaN
2,64,ArmenSoft,1.0,1.0000,male,NaN,NaN,NaN
3,128,okbookman,1.0,0.9945,male,0.998834,male,NaN
4,215,revitupmedia,1.0,1.0000,male,NaN,NaN,male


In [132]:
gd['username']=gd['username'].str.lower()

In [137]:
diff_2 = diff.join(gd.set_index('username'), how='inner')
diff_2.shape

(49, 9)

### conflicts

### you can check their profile online
* e.g.: the first one https://www.freelancer.com/u/anghelvioleta

In [138]:
diff_2

,desc_stat,label,user_id,freq,male.prob,fname.gender,male.prob2,fname.gender2,pic.gender
username,,,,,,,,,
anghelvioleta,1,0,1229717,1.0,0.000000,female,0.000000,female,NaN
anomakit,1,0,552690,3.0,0.000000,female,0.000000,female,NaN
anomakit,1,0,552690,3.0,0.000000,female,0.000000,female,NaN
anomakit,1,0,552690,3.0,0.000000,female,0.000000,female,NaN
anomakit,1,0,552690,3.0,0.000000,female,0.000000,female,NaN
babywebmaster,0,1,21290,2.0,0.996200,male,1.000000,male,NaN
bmihai10,1,0,226779,0.0,NaN,NaN,0.000000,female,NaN
bryeva,1,0,163045,2.0,0.000000,female,NaN,NaN,NaN
caileighfitz22,1,0,1231044,1.0,0.000000,female,NaN,NaN,NaN
